<a href="https://colab.research.google.com/github/microprediction/humpday/blob/main/humpday/objectives/discrete_objective_scratch_working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np 
q = np.array([22.21396024075403,
               18.450225620003916,
               13.091846318218286,
               8.005787316920557,
               2.134300511074848,
               3.544182319235291,
               35.01881404017196,
               10.13779260616384])
p =  np.array([0.01254655942360283,
            0.03733508812414016,
            0.08422143701988394,
            0.11380870336858935,
            0.4182923395784609,
            0.2009873711681508,
            0.027391228537457925,
            0.10541727277971412])

### Disclaimer. Don't use this ever for any reason. 




In [ ]:
!pip install pyportfolioopt

In [ ]:
from numpy import linalg as la
def fixcovariance(A):
    """Find the nearest positive-definite matrix to input

    A Python/Numpy port of John D'Errico's `nearestSPD` MATLAB code [1], which
    credits [2].

    [1] https://www.mathworks.com/matlabcentral/fileexchange/42885-nearestspd

    [2] N.J. Higham, "Computing a nearest symmetric positive semidefinite
    matrix" (1988): https://doi.org/10.1016/0024-3795(88)90223-6
    """
    def isPD(B):
      """Returns true when input is positive-definite, via Cholesky"""
      try:
          _ = la.cholesky(B)
          return True
      except la.LinAlgError:
          return False

    B = (A + A.T) / 2
    _, s, V = la.svd(B)

    H = np.dot(V.T, np.dot(np.diag(s), V))

    A2 = (B + H) / 2

    A3 = (A2 + A2.T) / 2

    if isPD(A3):
        return A3

    spacing = np.spacing(la.norm(A))
    # The above is different from [1]. It appears that MATLAB's `chol` Cholesky
    # decomposition will accept matrixes with exactly 0-eigenvalue, whereas
    # Numpy's will not. So where [1] uses `eps(mineig)` (where `eps` is Matlab
    # for `np.spacing`), we use the above definition. CAVEAT: our `spacing`
    # will be much larger than [1]'s `eps(mineig)`, since `mineig` is usually on
    # the order of 1e-16, and `eps(1e-16)` is on the order of 1e-34, whereas
    # `spacing` will, for Gaussian random matrixes of small dimension, be on
    # othe order of 1e-16. In practice, both ways converge, as the unit test
    # below suggests.
    I = np.eye(A.shape[0])
    k = 1
    while not isPD(A3):
        mineig = np.min(np.real(la.eigvals(A3)))
        A3 += I * (-mineig * k**2 + spacing)
        k += 1

    return A3


### Discrete allocation


In [ ]:
n = len(q)
X = -1.0*np.ones(shape=(n, n))
for j in range(n):
    X[j, j] = q[j] - 1.0
mu = np.dot(X, p)
X_centered = X - np.tile(mu[:, np.newaxis], [1, n])
p_eye = np.diag(p)
sgma = np.dot(np.dot(X_centered, p_eye), np.transpose(X_centered))
sgma_fixed = fixcovariance(sgma)
from pypfopt.efficient_frontier import EfficientFrontier
ef = EfficientFrontier(mu, sgma_fixed)
weights = ef.max_sharpe(risk_free_rate=0.0)
print(weights)


OrderedDict([(0, 0.0), (1, 0.0), (2, 0.5110295701668779), (3, 0.0), (4, 0.0), (5, 0.0), (6, 0.0), (7, 0.4889704298331223)])
